In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re
get_ipython().run_line_magic('matplotlib', 'inline')

#read in the data
tweet_data = pd.read_csv('../../data/raw-data/R_FBI_TEXT_DATA.csv')

#Column names
tweet_data.columns

#text data cleaning
#text data
tweet_data = tweet_data[['text']]

#replace # by space
def replace_hashtag(text):
    text = re.sub(r'#', ' ', text)
    return text

#remove punctuation
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+','', text) #removes numbers from text
    return text
tweet_data['clean text']=tweet_data['text'].apply(lambda x: remove_punct(x))

#tokenization
def tokenization(text):
    text = re.split('\W+', text) #splitting each sentence/ tweet into its individual words
    return text

tweet_data['Tweet_tokenized'] = tweet_data['clean text'].apply(lambda x: tokenization(x.lower()))

#remove retweets
tweet_data = tweet_data[~tweet_data['text'].str.contains('RT')]

#remove stopwords
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
stopwords
def remove_stopwords(text):
    text = [word for word in text if word not in stopwords]
    return text
   
tweet_data['Tweet_without_stop'] = tweet_data['Tweet_tokenized'].apply(lambda x: remove_stopwords(x))

#stemming
ps = nltk.PorterStemmer()
def stemming(text):
    text = [ps.stem(word) for word in text]
    return text
tweet_data['Tweet_stemmed'] = tweet_data['Tweet_without_stop'].apply(lambda x: stemming(x))

nltk.download('wordnet')
nltk.download('omw-1.4')
wordnet = nltk.WordNetLemmatizer()

#lemmatizations
def lemmatizer(text):
    text = [wordnet.lemmatize(word) for word in text]
    return text

tweet_data['Tweet_lemmatized'] = tweet_data['Tweet_without_stop'].apply(lambda x: lemmatizer(x))

#Count Vectorizer
from random import sample
clean_tweets=list(set(tweet_data['clean text']))
sample=sample(clean_tweets,20)
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sample)
vectorizer.get_feature_names()
count_vect_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

##exporting the cleaned data to a csv file
tweet_data.to_csv('../../data/modified-data/cleaned_fbi_text_data.csv', index=False)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chaitanyashekar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/chaitanyashekar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/chaitanyashekar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/Users/chaitanyashekar/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
